# Introduction

This notebook demonstrates our poetry chatbot, built using **LangChain**. The implementation consists of the following components:

- **`llm_interface.py`**: A wrapper around the HTTP APIs provided by `03_MyLLMServer.py`.
- **`secretory.py`**: The secretory agent responsible for intent recognition and poem retrieval.
- **`professor.py`**: The professor agent responsible for engaging in conversation with the user.

To run this notebook, ensure that `03_MyLLMServer.py` is running, either locally or on a public server.

Additionally, this notebook demonstrates using **`ipywidgets`** to create simple, interactive GUIs within notebooks.

In [ ]:
import json

import ipywidgets as widgets
from IPython.display import display

from professor import ProfessorAgent
from secretary import SecretaryAgent

In [ ]:
# Load config from a JSON file
with open("config.json", "r") as config_file:
    config = json.load(config_file)

# If using a public URL, set it below (default is http://localhost:7777)
# config["server_url"] = "https://few-cats-press.loca.lt"

# Initialize agents
professor_agent = ProfessorAgent(config)
secretary_agent = SecretaryAgent(config)

In [ ]:
# UI elements
user_input = widgets.Textarea(
    placeholder='Type your message here...',
    description='User:',
    layout={'width': '100%', 'height': '100px'}
)
submit_button = widgets.Button(
    description="Send",
    button_style='primary'
)
chat_output = widgets.Output(layout={'border': '1px solid black', 'padding': '10px', 'width': '100%'})
retrieval_output = widgets.Output(layout={'border': '1px solid black', 'padding': '10px', 'width': '100%'})


# Define the main function for handling the chat
def handle_chat(_):
    user_text = user_input.value.strip()
    if user_text.lower() == 'exit':
        with chat_output:
            print("Ending conversation. Goodbye!")
        user_input.value = ''
        user_input.disabled = True
        submit_button.disabled = True
        return

    # Clear outputs to refresh UI elements each time
    retrieval_output.clear_output()

    # Display user input in the chat history
    with chat_output:
        print(f"User: {user_text}")

    # Process retrieval based on intent
    retrieved_content = secretary_agent.retrieve(user_text)

    # Generate AI response using the professor agent
    ai_response = professor_agent.chat(user_text, retrieved_content)

    # Display retrieved content in the retrieval output box
    with retrieval_output:
        if professor_agent.last_retrieval:
            print(f"Retrieved Information:\n{professor_agent.last_retrieval}")
        else:
            print("No retrieval needed for this query.")

    # Display the AI response in the chat history
    with chat_output:
        print(f"AI: {ai_response}\n")

    # Clear the input box for the next user message
    user_input.value = ''

In [ ]:
# Greeting message in chat output
with chat_output:
    print("Welcome to the English Poetry Class! Type 'exit' to end the conversation.\n")

# Link the chat handler to the submit button
submit_button.on_click(handle_chat)

# Display the UI elements
display(user_input, submit_button, chat_output, retrieval_output)